In [16]:
import tempfile
tempdir = tempfile.mkdtemp()
from vaex.quick import *
import os
#table = vx.example()
table = vx.open("../data/Aq-A-2-999-shuffled-10percent.hdf5")
subspace = table("x", "y")


In [67]:
video_template = """<video controls>
 <source src="data:video/x-m4v;base64,{video_data_base64}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""
import base64
from IPython.display import display_html
def embed_mp4(video_data_binary):
    video_data_base64 = base64.b64encode(video_data_binary)
    html_code = video_template.format(**locals())
    display_html(html_code, raw=True)
    
def map_mp4(f, sequence, progress=True, embed=True, *args, **kwargs):
    !ffmpeg -version > /dev/null
    if _exit_code != 0:
        raise SystemError, "ffmpeg not found or exited with error %d" % _exit_code

    import tempfile
    tempdir = tempfile.mkdtemp()
    N = len(sequence)
    for i, element in enumerate(sequence):
        filename = os.path.join(tempdir, "frame-%05d.png" % i)
        if progress:
            print "\r", filename, (i+1), "out of", N, "% 5.1f%%" % (100.*i/float(N-1)),
        f(element, *args, **kwargs)
        pylab.savefig(filename)
        pylab.close()
    
    #!rm {tempdir}/movie.mp4 > /dev/null
    movie_filename = os.path.join(tempdir, "movie.mp4")
    !!ffmpeg -r 25 -i {tempdir}/frame-%5d.png -vcodec libx264 -crf 20 -threads 0 {movie_filename} > {tempdir}/ffmpeg_output.txt
    #if _exit_code != 0:
     #   raise SystemError, "ffmpeg exited with error %d, see %s for output" % (_exit_code, os.path.join(tempdir, "ffmpeg_output.txt"))
    if embed:
        embed_mp4(file(movie_filename).read())
    else:
        return movie_filename

In [68]:
limits = subspace.limits_sigma(sigmas=3, square=True)

def plot(fraction, subspace):
    (xmin, xmax), (ymin, ymax) = limits
    width = xmax - xmin
    height = ymax - ymin
    cx = (xmin+xmax)/2
    cy = (ymin+ymax)/2
    zoom_limits = (cx-width*fraction/2, cx+width*fraction/2), (cy-height*fraction/2, cy+height*fraction/2)
    subspace_bounded = subspace.bounded_by(zoom_limits)
    subspace_bounded.gridded().plot()

    fractions = 1/np.linspace(1, 2, 20)**0.5
map_mp4(plot, fractions, subspace=subspace)

/var/folders/vn/_rmzj8jd0215_g9yfrn8pmgm0000gn/T/tmpMUXY29/frame-00019.png 20 out of 20  100.0%

Your browser does not support the video tag.

In [75]:
z=vx.zeldovich(scale=100, t=1e1)
vx.set_log_level_warning()
z.add_virtual_column("xp", "x0+vx*t")
z.add_virtual_column("yp", "y0+vy*t")
def plot(i):
    z.variables["t"] = i
    z("xp", "yp").bounded_by([(-40, 40), (-40, 40)]).gridded().plot(cmap="afmhot")
map_mp4(plot, np.linspace(5, 15, 30))

/var/folders/vn/_rmzj8jd0215_g9yfrn8pmgm0000gn/T/tmpsp1jW7/frame-00029.png 30 out of 30  100.0%

Your browser does not support the video tag.